# Modelflow brings together a model and a Pandas DataFrame
**ModelFlow** is designed to combine a **model specification** with a **Pandas DataFrame** that contains the corresponding data.  
In this setup, each **column** of the DataFrame represents a model variable, while the **row index** represents time.  
This structure allows ModelFlow to evaluate and simulate the model directly on the data, period by period.  

While Pandas provides powerful tools for managing time series, it does not natively handle **lagged variables** or **simultaneous equation systems** in a convenient way.  
ModelFlow fills this gap by extending the DataFrame framework with capabilities for **lag and lead referencing**, **recursive and simultaneous solution algorithms**, and **automatic dependency handling** between variables.  

In this way, ModelFlow bridges the gap between **data storage** and **model computation**, making it possible to define, solve, and analyze complex dynamic systems directly within the familiar Python and Pandas environment.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from modelconstruct import Mexplode

## Specification of a model 
In its simplest form, the ModelFlow domain-specific language (DSL) consists of **formulas**, each defining a relationship between variables.  
However, ModelFlow also supports a **higher-level DSL** that extends this basic syntax with Python-based text processing features.  
This enables the user to **generate formulas programmatically**, allowing the model specification itself to be written at a **higher level of abstraction**.  

With this extended DSL, users can define **templates or patterns** for equations that apply across multiple entities — for example, all sectors in a macroeconomic model or all banks in a stress-testing framework — and then automatically expand them into a complete set of detailed, entity-specific equations.  

This approach dramatically reduces **boilerplate text** and improves **consistency** across the model.  
Instead of manually writing large blocks of repetitive equations, the modeler can rely on Python constructs such as loops, lists, and string formatting to generate the full specification dynamically and transparently.  

By integrating this higher-level DSL directly into the modeling framework, ModelFlow combines the **clarity and expressiveness** of a concise formula language with the **flexibility and automation** capabilities of Python — making it possible to specify large, structured models in a compact, readable, and reproducible way.


## Example Models  
To illustrate the use of ModelFlow and its formula-based DSL, we present two simple examples.  
The first is a small macroeconomic model describing the relationship between **GDP** and **consumption**.  
The second is a **debt dynamics** model that tracks the evolution of government debt over time.  
Together, they demonstrate how behavioral and accounting identities can be expressed concisely in ModelFlow’s syntax.  



### 1. GDP and Consumption Model  

This simple model links output $Y_t$ and consumption $C_t$ over time.  
Consumption depends on lagged disposable income, and output is determined by aggregate demand.  


\begin{align*}
C_t &= \alpha_0 + \alpha_1 Y_{t} + \alpha_2 Y_{t-1} + \varepsilon_t^C \\
Y_t &= C_t + I_t + G_t + X_t - M_t
\end{align*}


- $C_t$ – consumption at time $t$  
- $Y_t$ – output (GDP) at time $t$  
- $I_t, G_t, X_t, M_t$ – investment, government spending, exports, and imports  
- $\alpha_0, \alpha_1\ , \alpha_2$ – behavioral parameters  
- $\varepsilon_t^C$ – consumption shock  

This captures **lagged adjustment behavior** (consumption depends on past income) while treating the other demand components as **exogenous**. Also the model is contains **contemporaneous feedback loops** so it has to be solved as a system.   

To input this into ModelFlow the two formulas can be written as: 


```
Frml <stoc> C = a0 +  a1*Y + a2*Y(-1) + epsC $
Frml Y = C + I + G + X - M $
```

---

### 2. Debt Dynamics Model  

A second example tracks the evolution of government debt over time.  
Debt increases with the primary deficit and with interest payments on existing debt.  


\begin{align*}
D_t &= (1 + i_{t-1}) D_{t-1} + PD_t \\
PD_t &= G_t - T_t
\end{align*}

- $D_t$ – government debt at time $t$  
- $i_{t-1}$ – interest rate on debt from the previous period  
- $PD_t$ – primary deficit (spending minus taxes)  
- $G_t$ – government spending  
- $T_t$ – taxes  

Here, $D_t$ is **endogenous**, while $i_t$, $G_t$, and $T_t$ may be treated as **exogenous inputs** or determined by other model blocks.  

This simple dynamic ensures that current debt depends on **lagged debt** and **current fiscal flows**, a common structure in fiscal and macroeconomic models.  

To input this into ModelFlow the two formulas can be written as: 

```
Frml D  = (1 + i(-1))*D(-1) + PD $
Frml PD = G - T $
```



## A Model  

For the purpose of this note, a (simple) model like the two mentioned above can be thought of as a system of equations that links **endogenous** and **exogenous** variables over time.  

- **Endogenous variables** are those determined *within* the model — their values are the outcome of the relationships the model describes.  
- **Exogenous variables**, on the other hand, are determined *outside* the model — they are treated as given inputs or shocks that influence the endogenous variables but are not themselves affected by them.  

In time-based models, variables can appear with **lags** or **leads**.  
A *lag* refers to the value of a variable in a previous period — for example, \$y_{t-1}\$ is the value of \$y\$ one period before \$t\$.  
A *lead* refers to a future value — for example, \$y_{t+1}\$ represents the value of \$y\$ one period after \$t\$.  

Including lags allows the model to capture persistence and delayed effects in economic relationships, such as habit formation or gradual adjustment.  
In this note, only **lags** will be used, as they are sufficient for representing the backward-looking relationships typical of most empirical and accounting-based models.  
However, **ModelFlow** also allows the use of *leaded* variables, which are useful in models with rational or model-consistent expectations.  


Lets say we have:
 - **n**: number of endogenous variables  
 - **k**: number of exogenous variables  
 - **r**: maximum lag of endogenous variables  
 - **s**: maximum lag of exogenous variables  
 - **t**: time index (year, quarter, month, day, or any other unit)  


\begin{align*}
y_t^1  &= \;f^1(\; y_t^2,\ldots,y_t^n \;\;,\;\;y_{t-1}^1,\ldots,y_{t-r}^n\;\;,\;\;x_t^1,\ldots,x_t^k\;\;,\;\;x_{t-1}^1,\ldots,x_{t-s}^k \;) \\
y_t^2  &= \;f^2(\; y_t^1,\ldots,y_t^n \;\;,\;\;y_{t-1}^1,\ldots,y_{t-r}^n\;\;,\;\;  x_t^1,\ldots,x_t^k\;\;,\;\; x_{t-1}^1,\ldots,x_{t-s}^k \;) \\
       & \;\;\vdots \\
y_t^n  &= f^n\bigl(\; 
   \underbrace{\underbrace{y_t^1,\ldots,y_t^{n-1}}_{\text{current}} \;,\;
               \underbrace{y_{t-1}^1,\ldots,y_{t-r}^n}_{\text{lagged}}}_{\text{endogenous}},\;
   \underbrace{\underbrace{x_t^1,\ldots,x_t^k}_{\text{current}} \;,\;
               \underbrace{x_{t-1}^1,\ldots,x_{t-s}^k}_{\text{lagged}}}_{\text{exogenous}}
   \;\bigr)
\end{align*}


Sometime such models are implemented directly in a programming environment such as Python, Matlab, or even Excel, where they are both *specified* and *solved* in the same code.  

---

## Why Separate Specification and Implementation?  

Instead of mixing economic reasoning with programming details, a model can be written in a **domain-specific language (DSL)** designed specifically for economics and finance. Separating specification from implementation has several advantages:  

1. **Clarity and focus** – The specification remains close to the underlying economic logic, expressed in a concise and readable form. The modeler concentrates on *what the relationships are*, not on *how the computer solves them*.  

2. **Expressiveness** – The DSL uses syntax familiar to economists (equations, lags, variables), avoiding the clutter of general-purpose programming constructs.  

3. **Maintainability** – Models can be extended or updated without rewriting solution code. Changing an equation is as simple as editing one line in the specification.  

4. **Portability** – Once specified, the same model can be executed with different numerical engines or platforms without altering its economic content.  

5. **Transparency** – The separation makes models easier to review, validate, and communicate, since the specification reads like economic theory rather than computer code.  

This approach is common in several econometric systems such as **EViews**, **Dynare**, and **GEKKO**, which all use their own DSLs to describe models.  

**ModelFlow** follows a similar philosophy but with a key difference: it does not introduce a new language for data handling, visualization, or model analysis. Instead, ModelFlow relies on the **Python ecosystem** - primarily `pandas`, but also `networkx`, `sympy`, `numpy`, and other libraries - allowing the model specification to be combined seamlessly with powerful tools for data manipulation, result analysis, and visualization.  


## A Simple Specification Language  

In practice, the DSL for economic models is built around equation-like statements. For example:  
```
Frml <stoc> C = a0 + a1*Y(-1) + epsC $
Frml Y = C + I + G + X - M $

```

Here each formula (`Frml`) defines the relationship between variables in a way that mirrors how economists usually write them, with lags (`(-1)`) and simple algebraic notation.  

The optional `<tag>` is used for transforming the frml or to control solving - more on that later.

This kind of business-logic-oriented specification acts as the “blueprint” of the model, leaving the task of solving it to the underlying implementation engine.  


##  Mexplode – Expanding the Stripped-Down DSL 


The basic `Frml` statements form the **lowest level** of the ModelFlow domain-specific language (DSL), defining the direct relationships between variables. The basic `Frml` statements are transpiled into Python when the model is solved. 

To make the language more expressive, concise, and suitable for large, structured models, ModelFlow uses a preprocessor called **`Mexplode`** to extends this foundation with several higher-level constructs:  

- **Lists and sublists** – to organize entities such as banks, sectors, or countries, and automatically expand equations across these groups.  
- **Do ... enddo loops** – to generate sets of equations programmatically over the members of a list or over a specified range.  
- **Sum operator** – to express aggregation directly within equations by summing over lists or sublists.  
- **Doable equations** – shorthand notation combining loops and summation, simplifying the definition of repeated or aggregated relationships.  
- **Stripped-down syntax** – allows equations to be written without explicit `Frml` and `$` markers, resulting in cleaner, more compact specifications.  
- **Model specifications embedded in Markdown** – each line of model code starts with `>`, making them visually distinct and easily readable within documents. Documentation and explanations can be formatted nicely alongside the code using Markdown.  

Together, these features make it possible to write models at a **higher level of abstraction**, reducing repetition and improving consistency across model components.  
Instead of defining each equation manually, the modeler can use lists, loops, and operators to express economic logic once and have ModelFlow automatically generate all the necessary detailed equations.  
This design allows the DSL to scale from small illustrative examples to large, multi-entity macroeconomic and financial models while maintaining clarity and transparency.  


### Mexplode Properties  


:::{index} single: Box 1 Mexplode Properties
:::

:::{admonition} Box 1 Mexplode Properties

**For use by the very interested modeller**, the `Mexplode` dataclass stores the full transformation state of a model as it moves from the stripped-down DSL to the fully expanded, normalized set of `Frml` statements ready for ModelFlow.  
Each field corresponds to a specific stage in this expansion and normalization process.  

| **Property** | **Type** | **Description** |
|:---------------|-----------|:-----------------|
| `original_statements` | `str` | Input model specification written in the stripped-down DSL. |
| `normal_frml` | `str` | Output containing the normalized model equations. |
| `normal_main` | `str` | Normalized `Frml` statements for the main model equations. |
| `normal_fit` | `str` | Normalized `Frml` statements for fitted-value calculations. |
| `normal_calc_add` | `str` | Normalized `Frml` statements for add-factor calculations. |
| `clean_frml_statements` | `str` | Intermediate version with properly formatted `Frml` lines and structured list syntax. |
| `post_doable` | `str` | Model after expansion of all `doable` equations. |
| `post_do` | `str` | Model after expansion of explicit `do ... enddo` loops. |
| `post_sum` | `str` | Model after expansion of all summation (`sum`) operators. |
| `expanded_frml` | `str` | Fully expanded set of executable `Frml` equations. |
| `normal_expressions` | `List[Any]` | List of parsed normalized expression objects. |
| `list_specification` | `str` | String representation of all list definitions found in the model. |
| `modellist` | `str` | Dictionary-like string of defined lists and their attributes. |
| `funks` | `List[Any]` | List of user-defined functions that can be referenced in model formulas. |

These internal fields allow the modeller to follow, in detail, how the stripped-down DSL is gradually transformed into executable ModelFlow syntax.  
They provide a complete diagnostic view of the intermediate stages—useful for debugging, educational purposes, or extending ModelFlow’s preprocessing pipeline.  

The higher-level properties such as **`show`** and **`mmodel`**—described in the main `Mexplode` section—build on this internal structure to present the final expanded model or to produce a fully functional ModelFlow model instance.

:::


### Lists  

Among the higher-level constructs, **lists** and **sublists** are central to making ModelFlow scalable and expressive.  
They allow the modeler to define **groups of entities** (such as banks, portfolios, sectors, or countries) and to attach **attributes** or **hierarchies** to those entities.  
This enables equations to be written once as templates and then **expanded automatically** across all relevant list members.  

Sublists can be used to define logical relationships between entities — for example, linking each bank to its home country, or associating portfolios with regulatory categories.  
These structures make it possible to organize models systematically and to maintain consistency across entities that share the same behavioral equations.  

This approach eliminates redundancy and manual repetition: instead of writing dozens or hundreds of similar equations, the modeler defines a **single equation pattern**, and ModelFlow expands it dynamically over the members of a list.  

This capability is especially valuable in:  

- **Stress-testing models** – where many banks, each with multiple portfolios, must be represented consistently.  
- **Macroeconomic models** – where sectors of the economy (households, firms, government, rest of the world, etc.) share common behavioral structures.  
- **Cross-country or panel models** – where the same specification is applied across multiple countries or regions.  

#### Syntax  

List definitions in the ModelFlow DSL follow a simple and consistent format:  

- Each list **starts** with the keyword `list`.    
- It can contain one or more **sublists**, separated by `/`.  
- Each sublist **starts** with a `name:` followed by its members.  


Below are a few examples of list definitions:  

In [38]:
lists ='''
## Lists used in the next examples 
The folloving lists are only examples 

First a list of banks and of the home country for each bank 
>list BANKS = BANKS    : Danske  Nordea Ibs /
>             bankcountry : denmark sweden denmark  /
>             include   :   0        1       1

A list of countries 
>list country = country : denmark sweden 

A list of different portefolios and of the regulary treatment 
>list ports = ports  : households   NFC    RE  sov /
>              reg  : AIRB  AIRB   STA  STA 


'''

Meaning


- `list BANKS = ...` defines a top-level list of banks with an **sublist** `bankcountry` that specifies where each bank belongs.  
- `list country = ...` creates a country list.  
- `list ports = ...` groups portfolios (`households`, `NFC`, `RE`, `sov`) and creates **sublists** with regulatory categories (`AIRB`, `STA`).  

So the list statement starts with the keyword `list` and a `list name =`. Sublists are seperated by / and each starts with a `sublist name:` 

It makes sense to give the first sublist the same name as the list name. 

The benefits of lists: 

- **Compactness** – groups can be declared once and reused throughout the model.  
- **Flexibility** – adding or removing members requires only updating the list, not the equations.  
- **Transparency** – clear structure makes it easy to see which entities and attributes the model covers.  

By defining lists in this way, ModelFlow provides a structured layer between the economic logic and the generated equations, enabling users to manage complex multi-entity models efficiently and transparently.

### Do ... Enddo Loops  

The **`do ... enddo`** construct allows repetitive patterns of equations to be generated automatically.  
When used together with **lists**, it enables the modeler to create a complete set of equations for every member of a list without having to write each one manually.  

A `do` block defines a **loop** over the members of a specified list (for example, all banks, sectors, or countries).  
Within the block, the loop variable can be referenced directly inside formulas — ModelFlow will automatically substitute the appropriate member name during expansion.  

This mechanism is essential for large-scale models where the same behavioral relationships apply to multiple entities.  
It ensures **consistency**, reduces **repetition**, and makes model specifications more **maintainable** and **transparent**.  

#### Syntax  

The syntax of a `do` block is straightforward:  

- The loop *starts* with: $\textbf{do } \texttt{list name}\;\;  \underbrace{ \texttt{sublist name}=\texttt{value}}_{\text{optional}} $
- It *ends* with: $\textbf{enddo}$  

One or more Frml statements — and possibly nested do ... enddo blocks — are written inside the loop. 

**Mexplode** automatically expands each Frml statement for every member of the specified list that satisfies the optional filter.

#### Example  
If the Python variable lists already defines some model lists, it can be combined with an example model specification and passed to `Mexplode` for processing.

The `show` property can then be used to display the resulting DSL equations after expansion.

In [4]:
list_example = Mexplode(lists +
''' 
## Loop over all banks
>do banks 
Calculate profit in each bank 
>     profit__{banks} = revenue__{banks} - expenses__{banks} 
Terminate the loop 
> enddo  
''')
list_example.show

FRML <> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <> PROFIT__NORDEA = REVENUE__NORDEA-EXPENSES__NORDEA $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $


`.showmd` Will render whole Markdown text

In [5]:
list_example.render


## Lists used in the next examples 
The folloving lists are only examples 

First a list of banks and of the home country for each bank 

```text
>list BANKS = BANKS    : Danske  Nordea Ibs /
>             bankcountry : denmark sweden denmark  
```

A list of countries 

```text
>list country = country : denmark sweden 
```

A list of different portefolios and of the regulary treatment 

```text
>list ports = ports  : households   NFC    RE  sov /
>              reg  : AIRB  AIRB   STA  STA 
```


 
## Loop over all banks

```text
>do banks 
```
Calculate profit in each bank 

```text
>     profit__{banks} = revenue__{banks} - expenses__{banks} 
```
Terminate the loop 

```text
> enddo  
```

### Conditional Loops
The do statement can also include conditions that restrict the loop to members of a list matching specific attribute values.
This is written using the syntax do <list> <sublist>=<value>.

For example, suppose the list BANKS includes a sublist bankcountry,
then the following block will only expand for banks located in Denmark:

In [35]:
Mexplode(lists +
''' 
>do banks bankcountry = denmark
>     profit__{banks} = revenue__{banks} - expenses__{banks} 
> enddo  
''').show

FRML <> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $


ModelFlow will expand this block only for banks whose bankcountry attribute equals denmark.

This conditional looping makes it possible to write targeted relationships for specific subsets of entities 
for instance, defining different equations for domestic and foreign banks, or applying alternative behavioral rules to different sectors or regions.

By combining lists, sublists, and conditional do loops, ModelFlow allows highly structured models to be specified compactly while preserving clarity and flexibility.

### Doable Equations  

While `do ... enddo` loops provide an explicit way to expand formulas across members of a list,  
the **`doable`** construct offers a more concise and automatic alternative.  

A **doable equation** is a *high-level shorthand* that combines the functionality of loops and summations without requiring the user to write repetitive loop structures.  
Instead of manually specifying `do` blocks, ModelFlow can **detect the relevant lists automatically** by inspecting the **first** variable names in the equation.  

Variables that contain double underscores (`__`) indicate **list dimensions**.  
For example, in `profit__{banks}`, ModelFlow recognizes that `profit` is a variable defined over the list `BANKS`.  
Similarly, in `loss__{banks}__{ports}`, the two underscores identify two list dimensions — `BANKS` and `PORTS`.  


:::{index} single: Box 2 Doable summary
:::

:::{admonition} Box 2 Doable summary

$$
\texttt{doable} \;
\underbrace{\langle tag \rangle}_{\text{Tag}} \;
\underbrace{[list= sublist=value,\ldots]}_{\text{Optional Conditions}} \;
\underbrace{%
   \underbrace{\texttt{VARIABLE}}_{\text{LHS Expression}}
   \underbrace{\_\_\{list1\}\_\_\{list2\}...}_{\text{LHS Dimensions}}
}_{\text{LHS}} \;
= \;
\underbrace{\texttt{Expression including sublists}}_{\text{RHS Expression}}
$$

:::


When ModelFlow encounters a `doable` equation such as:  

In [7]:
Mexplode(lists + '''
>doable profit__{banks} = revenue_{banks} - expenses_{banks}
''').show   

FRML <> PROFIT__DANSKE = REVENUE_DANSKE-EXPENSES_DANSKE $
FRML <> PROFIT__NORDEA = REVENUE_NORDEA-EXPENSES_NORDEA $
FRML <> PROFIT__IBS = REVENUE_IBS-EXPENSES_IBS $


it automatically expands the formula across all members of the list BANKS — just as a do BANKS ... enddo loop would.

Thus, the doable keyword eliminates the need for explicit looping syntax, making the specification both shorter and easier to read.

Doable equations can also include conditions and tags in the same way as do loops.
For example:

In [37]:
Mexplode(lists+'>doable [banks= bankcountry=denmark] profit__{banks}=revenue__{banks}-expenses__{banks}').show 

FRML <> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $


Expands only for banks where the sublist bankcountry equals denmark.

In [39]:
Mexplode(lists+'>doable [banks=include] profit__{banks}=revenue__{banks}-expenses__{banks}').show 

FRML <> PROFIT__NORDEA = REVENUE__NORDEA-EXPENSES__NORDEA $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $


Expands only for banks where the sublist include has the value 1

This mechanism is particularly powerful when dealing with complex, multi-dimensional systems — such as stress-test models with many banks and portfolios —
since the modeler can write compact, pattern-based equations that ModelFlow expands automatically by detecting all relevant list dimensions.

In short, doable equations serve as an elegant, high-level syntax that:

 - Avoids the boilerplate of do ... enddo structures,

 - Automatically determines which lists apply by analyzi ng variable dimensions, and

 - Keeps model specifications concise, readable, and scalable.

## Sum of variables 
To sum over lists the sum statement en expressions can be used. it will generate a sum over the list and potential filtered by a condition 

In [9]:
Mexplode(lists+'>loss_danske =  sum(ports,loss__danske__{ports}))').show 

FRML <> LOSS_DANSKE = (LOSS__DANSKE__HOUSEHOLDS+LOSS__DANSKE__NFC+LOSS__DANSKE__RE+LOSS__DANSKE__SOV)) $


### Nested sums

In [10]:
Mexplode(lists+'>loss_total =  sum(banks,sum(ports,loss__{banks}__{ports}))').show 

FRML <> LOSS_TOTAL = ((LOSS__DANSKE__HOUSEHOLDS+LOSS__DANSKE__NFC+LOSS__DANSKE__RE+LOSS__DANSKE__SOV)+(LOSS__NORDEA__HOUSEHOLDS+LOSS__NORDEA__NFC+LOSS__NORDEA__RE+LOSS__NORDEA__SOV)+(LOSS__IBS__HOUSEHOLDS+LOSS__IBS__NFC+LOSS__IBS__RE+LOSS__IBS__SOV)) $


### Conditional sum
Here the total sum for all banks for portefolios with reg=sta  

In [11]:
Mexplode(lists+ '>loss_total_sta =  sum(banks,sum(ports reg=STA,loss__{banks}__{ports}))').show

FRML <> LOSS_TOTAL_STA = ((LOSS__DANSKE__RE+LOSS__DANSKE__SOV)+(LOSS__NORDEA__RE+LOSS__NORDEA__SOV)+(LOSS__IBS__RE+LOSS__IBS__SOV)) $


In [45]:
Mexplode(lists+ '>loss_total_sta =  sum(banks include,sum(ports reg=STA,loss__{banks}__{ports}))').show


FRML <> LOSS_TOTAL_STA = ((LOSS__NORDEA__RE+LOSS__NORDEA__SOV)+(LOSS__IBS__RE+LOSS__IBS__SOV)) $


## Combining doable and sum
In `doable` to create a sum of all the calculated Conditional the tag sum can be used. This will generate a sum of all the calculated values in the statement. `<sum=text>` will give dimension name to the result. If no text is given it will be  called sum.

In [12]:
Mexplode(lists+
'>doable <sum=total>  profit__{banks}=revenue__{banks}-expenses__{banks}'
        ).show

FRML <SUM=TOTAL> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <SUM=TOTAL> PROFIT__NORDEA = REVENUE__NORDEA-EXPENSES__NORDEA $
FRML <SUM=TOTAL> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $
FRML <SUM=TOTAL> PROFIT__TOTAL = (PROFIT__DANSKE+PROFIT__NORDEA+PROFIT__IBS) $


And with conditions it works to

In [13]:
exploded = Mexplode(lists+
'>doable <sum=total> [banks bankcountry=denmark] profit__{banks}=revenue__{banks}-expenses__{banks}'
        )
exploded.show

FRML <SUM=TOTAL> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <SUM=TOTAL> PROFIT__NORDEA = REVENUE__NORDEA-EXPENSES__NORDEA $
FRML <SUM=TOTAL> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $
FRML <SUM=TOTAL> PROFIT__TOTAL = (PROFIT__DANSKE+PROFIT__NORDEA+PROFIT__IBS) $


### `.draw` — Displaying the Model’s Causal Structure  

The **`.draw`** method visualizes the **causal structure** of a model as defined by its equations.  
It provides an intuitive graph of dependencies between variables, helping to understand the logical flow and interconnections within the model.  

:::{warning}
Use this feature only for **small models** — for larger models, the resulting diagram can become cluttered.
:::


In [14]:
exploded.draw

The **`Mexplode.draw`** property is a convenient wrapper around **`Mexplode.mmodel.drawmodel()`**.  
It first accesses the **`modelclass.model`** instance defined by the DSL, then generates a graphical representation of the entire model.  

All methods available to the underlying model instance can be used — for example, the **`.tracedep`** method, which allows tracing the dependencies of a single variables within the model.  


In [15]:
exploded.mmodel.PROFIT__DANSKE.tracepre() 

## Tags and Formula Decoration  

As mentioned earlier, the user should focus primarily on defining the **economic relationships** themselves.  
However, in some cases it can be useful to **decorate** formulas to enable specific **modifications or behaviors** during calculations.  
This is achieved by **tagging** the formula.  

If we start with this simple formula and no tags. The formula will just be translated to a simple DSL formula:

In [16]:
Mexplode('>a = b').show

FRML <> A = B $


### Enable fixing the value of a a variable 
When the value of a variable is known for certain time periods, it can be fixed by tagging its formula with exo.
This is especially useful in forecasting exercises, where newly published statistics are available and should override the model’s computed values.

When a formula is tagged with exo, two additional variables are automatically created:

 - <variable name>_X — the fixed (exogenous) value to be used.

 - <variable name>_D — a dummy variable that equals 1 in the periods where the fixed value should replace the model-generated one.
        

In [17]:
Mexplode('><exo> a = b').show

FRML <EXO> A = (B)* (1-A_D)+ A_X*A_D $


his means that:

 - When `A_D = 0`, the model uses the endogenous calculation A = B.

 - When `A_D = 1`, the model instead uses the fixed value A = A_X.

This mechanism allows the modeller to incorporate known or published values seamlessly, without modifying the model’s structure.

The ModelFlow model instance also includes convenient helper methods for managing these exogenous values — for example, to set, update, or release fixed variables during forecasting or simulation exercises.

### Include add factors
In some it can be useful to be able to adjust the formula in order to make an experiment or to incorporate factors outside the model. Sometime called **add factors** or **expert judgements** To make life easy this can be done by using the tag `add` also the tag `add_suffic=` can be used to change the suffix for the add factor from the default `_A`

In [18]:
Mexplode('><add> a = b').show

FRML <ADD> A = (B + A_A)                                $

FRML <CALC_ADD_FACTOR>  A_A = (A) - (B) $


### Reverse Calculation of Add Factors  

A special `Frml` tagged with **`CALC_ADD_FACTOR`** is automatically generated by **Mexplode**.  
This formula is used by the **ModelFlow model instance** to build a specialized *calculation model* that computes the **add factors** — the residual adjustments needed to reconcile actual or fixed values with the model’s baseline equations.  

On its own, this feature is not particularly useful, but when **combined with the `exo` tag**, it becomes powerful.  
Together, they allow the model to **retain its marginal (behavioral) properties** even when some variables have **fixed values**.  

After the add factors have been computed, the modeller can **reset the corresponding `_D` dummies to 0**, effectively releasing the fixed variables.  
The model can then be used for **experiments, policy simulations, or shock analyses**, while the previously calculated add factors ensure internal consistency and preserve the model’s original fit to observed data.  


In [19]:
Mexplode('><exo,add> a = b').show

FRML <EXO,ADD> A = (B + A_A)* (1-A_D)+ A_X*A_D  $

FRML <CALC_ADD_FACTOR>  A_A = (A) - (B) $


### Naming of Add Factors  

If the modeller prefers a **different naming convention** for add factors, this can be achieved by including the tag **`add_suffix=`** in the formula specification.  
For example:

In [20]:
Mexplode('><add,add_suffix=__J> a = b').show

FRML <ADD,ADD_SUFFIX=__J> A = (B + A__J)                                $

FRML <CALC_ADD_FACTOR>  A__J = (A) - (B) $


generate add factors named with the suffix `_J` instead of the default `_A`.  

### Including Calculated Fitted Values  

When **add factors** or **fixed values** are used, it may be useful to compute the value implied by the **original fitted formula** — that is, the model’s unadjusted behavioral relationship.  
This can be achieved by tagging the formula with **`FIT`**.  

When the `FIT` tag is applied, an additional formula is automatically created.  
This formula calculates a new variable with the suffix **`_FITTED`**, representing the value generated by the **original model equation** (before the inclusion of add factors or fixed values).  

This is helpful for comparing the model’s baseline predictions with adjusted or constrained outcomes, especially in forecasting and diagnostic analysis.  


In [21]:
Mexplode('><exo,add,fit> a = b').show

FRML <EXO,ADD,FIT> A = (B + A_A)* (1-A_D)+ A_X*A_D  $
FRML <FIT>  A_FITTED = B $ 
FRML <CALC_ADD_FACTOR>  A_A = (A) - (B) $


In [22]:
Mexplode(lists+
'>doable <sum=total,exo,fit,add>  profit__{banks}=revenue__{banks}-expenses__{banks}'
        ).show

FRML <SUM=TOTAL,EXO,FIT,ADD> PROFIT__DANSKE = (REVENUE__DANSKE-EXPENSES__DANSKE + PROFIT__DANSKE_A)* (1-PROFIT__DANSKE_D)+ PROFIT__DANSKE_X*PROFIT__DANSKE_D  $
FRML <SUM=TOTAL,EXO,FIT,ADD> PROFIT__NORDEA = (REVENUE__NORDEA-EXPENSES__NORDEA + PROFIT__NORDEA_A)* (1-PROFIT__NORDEA_D)+ PROFIT__NORDEA_X*PROFIT__NORDEA_D  $
FRML <SUM=TOTAL,EXO,FIT,ADD> PROFIT__IBS = (REVENUE__IBS-EXPENSES__IBS + PROFIT__IBS_A)* (1-PROFIT__IBS_D)+ PROFIT__IBS_X*PROFIT__IBS_D  $
FRML <SUM=TOTAL,EXO,FIT,ADD> PROFIT__TOTAL = ((PROFIT__DANSKE+PROFIT__NORDEA+PROFIT__IBS) + PROFIT__TOTAL_A)* (1-PROFIT__TOTAL_D)+ PROFIT__TOTAL_X*PROFIT__TOTAL_D  $
FRML <FIT>  PROFIT__DANSKE_FITTED = REVENUE__DANSKE-EXPENSES__DANSKE $ 
FRML <FIT>  PROFIT__NORDEA_FITTED = REVENUE__NORDEA-EXPENSES__NORDEA $ 
FRML <FIT>  PROFIT__IBS_FITTED = REVENUE__IBS-EXPENSES__IBS $ 
FRML <FIT>  PROFIT__TOTAL_FITTED = (PROFIT__DANSKE+PROFIT__NORDEA+PROFIT__IBS) $ 
FRML <CALC_ADD_FACTOR>  PROFIT__DANSKE_A = (PROFIT__DANSKE) - (REVENUE__DANSKE-EXPENSE

## Using `replacements=` or `str.replace()` to simplify multi-dimensional formulas

When a formula includes several dimensions (e.g., `banks`, `country`, `ports`), variable names can become long and repetitive.  
To keep model definitions compact and readable, you can define formulas with **placeholders** (such as `__dim`) and then insert the actual dimension names automatically.

There are now **two ways** to do this:

1. Use Python’s built-in **`str.replace()`** directly in the formula string.  
2. Use the **`replacements=` argument** in `Mexplode`, which applies the same logic internally`.

The replacements argument accepts either **a single (pattern, replacement) tuple**
or **a list of such tuples** for multiple substitutions.


### 💡 Why this helps

- **Cleaner input:** Focus on the formula’s logic instead of long dimension names.  
- **Consistency:** Reuse the same placeholder pattern across multiple equations.  
- **Maintainability:** Change dimensional naming once in the `replacements` list instead of editing all formulas.  
- **Convenience:** The `replacements=` argument avoids having to call `.replace()` manually.


### 🧮 Example

Below, the same model is defined in two equivalent ways — first using `.replace()`, and then using the  `replacements=` argument.

In [23]:
# Example 1 — Using str.replace() manually
Mexplode(lists +
    '>doable <> loss__dim = portefolio__dim * pd__dim * lgd__dim'
        .replace('__dim', '__{banks}__{country}__{ports}')
).show

FRML <> LOSS__DANSKE__DENMARK__HOUSEHOLDS = PORTEFOLIO__DANSKE__DENMARK__HOUSEHOLDS*PD__DANSKE__DENMARK__HOUSEHOLDS*LGD__DANSKE__DENMARK__HOUSEHOLDS $
FRML <> LOSS__DANSKE__DENMARK__NFC = PORTEFOLIO__DANSKE__DENMARK__NFC*PD__DANSKE__DENMARK__NFC*LGD__DANSKE__DENMARK__NFC $
FRML <> LOSS__DANSKE__DENMARK__RE = PORTEFOLIO__DANSKE__DENMARK__RE*PD__DANSKE__DENMARK__RE*LGD__DANSKE__DENMARK__RE $
FRML <> LOSS__DANSKE__DENMARK__SOV = PORTEFOLIO__DANSKE__DENMARK__SOV*PD__DANSKE__DENMARK__SOV*LGD__DANSKE__DENMARK__SOV $
FRML <> LOSS__DANSKE__SWEDEN__HOUSEHOLDS = PORTEFOLIO__DANSKE__SWEDEN__HOUSEHOLDS*PD__DANSKE__SWEDEN__HOUSEHOLDS*LGD__DANSKE__SWEDEN__HOUSEHOLDS $
FRML <> LOSS__DANSKE__SWEDEN__NFC = PORTEFOLIO__DANSKE__SWEDEN__NFC*PD__DANSKE__SWEDEN__NFC*LGD__DANSKE__SWEDEN__NFC $
FRML <> LOSS__DANSKE__SWEDEN__RE = PORTEFOLIO__DANSKE__SWEDEN__RE*PD__DANSKE__SWEDEN__RE*LGD__DANSKE__SWEDEN__RE $
FRML <> LOSS__DANSKE__SWEDEN__SOV = PORTEFOLIO__DANSKE__SWEDEN__SOV*PD__DANSKE__SWEDEN__SOV*LGD__DANSKE

In [24]:
repl=('__dim', '__{banks}__{country}__{ports}')

Mexplode(lists+
'>doable <>  loss__dim = portefolio__dim * pd__dim*lgd__dim',        
replacements=repl      ).show

FRML <> LOSS__DANSKE__DENMARK__HOUSEHOLDS = PORTEFOLIO__DANSKE__DENMARK__HOUSEHOLDS*PD__DANSKE__DENMARK__HOUSEHOLDS*LGD__DANSKE__DENMARK__HOUSEHOLDS $
FRML <> LOSS__DANSKE__DENMARK__NFC = PORTEFOLIO__DANSKE__DENMARK__NFC*PD__DANSKE__DENMARK__NFC*LGD__DANSKE__DENMARK__NFC $
FRML <> LOSS__DANSKE__DENMARK__RE = PORTEFOLIO__DANSKE__DENMARK__RE*PD__DANSKE__DENMARK__RE*LGD__DANSKE__DENMARK__RE $
FRML <> LOSS__DANSKE__DENMARK__SOV = PORTEFOLIO__DANSKE__DENMARK__SOV*PD__DANSKE__DENMARK__SOV*LGD__DANSKE__DENMARK__SOV $
FRML <> LOSS__DANSKE__SWEDEN__HOUSEHOLDS = PORTEFOLIO__DANSKE__SWEDEN__HOUSEHOLDS*PD__DANSKE__SWEDEN__HOUSEHOLDS*LGD__DANSKE__SWEDEN__HOUSEHOLDS $
FRML <> LOSS__DANSKE__SWEDEN__NFC = PORTEFOLIO__DANSKE__SWEDEN__NFC*PD__DANSKE__SWEDEN__NFC*LGD__DANSKE__SWEDEN__NFC $
FRML <> LOSS__DANSKE__SWEDEN__RE = PORTEFOLIO__DANSKE__SWEDEN__RE*PD__DANSKE__SWEDEN__RE*LGD__DANSKE__SWEDEN__RE $
FRML <> LOSS__DANSKE__SWEDEN__SOV = PORTEFOLIO__DANSKE__SWEDEN__SOV*PD__DANSKE__SWEDEN__SOV*LGD__DANSKE

## Using `list_defs=` to define lists in `Mexplode`

In many cases, it can be convenient to provide **list definitions** as an argument to `Mexplode` rather than embedding them directly into the model specification string.  

This approach helps separate the **technical structure** (lists, dimensions, replacements) from the **economic logic** (the actual formulas), making your model files cleaner and easier to read.

---

### 💡 When to use `list_defs=`

- When several submodels share the **same list definitions**.  
- When you want to keep the **economic equations visible**.  

---

### ⚙️ How it works

You can pass a string containing the list definitions directly to `Mexplode` using the `list_defs=` argument.  
`Mexplode` automatically prepends these lists to the model specification before parsing and normalization.



In [25]:
Mexplode(lists +'>doable profit__{banks} = revenue_{banks} - expenses_{banks}').show   

FRML <> PROFIT__DANSKE = REVENUE_DANSKE-EXPENSES_DANSKE $
FRML <> PROFIT__NORDEA = REVENUE_NORDEA-EXPENSES_NORDEA $
FRML <> PROFIT__IBS = REVENUE_IBS-EXPENSES_IBS $


In [26]:
Mexplode('>doable profit__{banks} = revenue_{banks} - expenses_{banks}',list_defs=lists).show   

FRML <> PROFIT__DANSKE = REVENUE_DANSKE-EXPENSES_DANSKE $
FRML <> PROFIT__NORDEA = REVENUE_NORDEA-EXPENSES_NORDEA $
FRML <> PROFIT__IBS = REVENUE_IBS-EXPENSES_IBS $


## Using `functools.partial` to create simplified `Mexplode` constructors

When building many model components that share the same configuration — such as common `list_defs`, `replacements`, or `funks` —  
you can simplify your workflow by creating a **pre-configured version** of `Mexplode` using `functools.partial`.

This avoids repeating the same arguments for every model block and makes your code cleaner and more consistent.

---

### 💡 Why this helps

- **Less repetition:** No need to retype the same `list_defs` or `replacements` each time.  
- **Consistency:** All model blocks use identical definitions and structural setup.  
- **Convenience:** The resulting constructor behaves exactly like `Mexplode`, but with defaults already filled in.  

---

Example: 

In [27]:
from functools import partial
SMexplode = partial(Mexplode,list_defs=lists, replacements=('__dim', '__{banks}__{country}__{ports}'))

SMexplode('>doable <>  loss__dim = portefolio__dim * pd__dim*lgd__dim').show

FRML <> LOSS__DANSKE__DENMARK__HOUSEHOLDS = PORTEFOLIO__DANSKE__DENMARK__HOUSEHOLDS*PD__DANSKE__DENMARK__HOUSEHOLDS*LGD__DANSKE__DENMARK__HOUSEHOLDS $
FRML <> LOSS__DANSKE__DENMARK__NFC = PORTEFOLIO__DANSKE__DENMARK__NFC*PD__DANSKE__DENMARK__NFC*LGD__DANSKE__DENMARK__NFC $
FRML <> LOSS__DANSKE__DENMARK__RE = PORTEFOLIO__DANSKE__DENMARK__RE*PD__DANSKE__DENMARK__RE*LGD__DANSKE__DENMARK__RE $
FRML <> LOSS__DANSKE__DENMARK__SOV = PORTEFOLIO__DANSKE__DENMARK__SOV*PD__DANSKE__DENMARK__SOV*LGD__DANSKE__DENMARK__SOV $
FRML <> LOSS__DANSKE__SWEDEN__HOUSEHOLDS = PORTEFOLIO__DANSKE__SWEDEN__HOUSEHOLDS*PD__DANSKE__SWEDEN__HOUSEHOLDS*LGD__DANSKE__SWEDEN__HOUSEHOLDS $
FRML <> LOSS__DANSKE__SWEDEN__NFC = PORTEFOLIO__DANSKE__SWEDEN__NFC*PD__DANSKE__SWEDEN__NFC*LGD__DANSKE__SWEDEN__NFC $
FRML <> LOSS__DANSKE__SWEDEN__RE = PORTEFOLIO__DANSKE__SWEDEN__RE*PD__DANSKE__SWEDEN__RE*LGD__DANSKE__SWEDEN__RE $
FRML <> LOSS__DANSKE__SWEDEN__SOV = PORTEFOLIO__DANSKE__SWEDEN__SOV*PD__DANSKE__SWEDEN__SOV*LGD__DANSKE

## Building modular models by combining smaller formula groups

Complex models often consist of multiple logical components — such as behavioral equations, accounting identities, or sector-specific submodels.  
Instead of defining one large block of equations in a single string, it’s often more efficient to **specify smaller groups of formulas** and then combine them into a complete model using the `+` operator.

---

### 💡 Why this approach is beneficial

- **Modularity:** Each `Mexplode` object can represent a distinct part of the model — for example, a banking block, a household sector, or a price system.  
- **Clarity:** Smaller, self-contained equation groups are easier to read, debug, and test independently.  
- **Reusability:** You can reuse model fragments across projects or scenarios by simply combining them differently.  
- **Incremental development:** Start with a minimal subset of formulas, verify behavior, and then expand the model progressively.

---

### ⚙️ How it works

The `+` operator merges multiple `Mexplode` objects into a single `Lexplode` container, which behaves like a unified model.  
This aggregation does not recompute anything — it simply combines their normalized forms, making it fast and memory-efficient.

Below, we define two small model blocks:


Each block is defined as an independent `Mexplode`, and then we combine them using `+`.  
This creates a single integrated model (`Lexplode`) that can be displayed or solved as one unit.

In larger models, it’s good practice to define separate logical components (blocks) for different economic or financial relationships.  
Here, we define three related parts of a simple very stylized credit risk model:

1. **PD update block** – describes how each bank’s probability of default (`PD`) evolves.  
2. **Expected Loss block** – calculates expected losses as a function of exposure, PD, and loss given default (`LGD`).
3. **Bank Expected Loss block** – calculates expected losses for each bank.

Each block is created as an independent `Mexplode` object.  
We then combine them with the `+` operator to form a complete integrated model (`Lexplode`), which can be solved or analyzed as one structure.


In [28]:
# --- Define three small, modular formula groups ---
# first define the replacements 
repl = ('__dim', '__{banks}__{country}__{ports}')
# --- Block 1: PD update equations ---
pd_update_block = Mexplode(lists+
'>doable <> pd__dim = 0.02 + 0.8 * pd__dim(-1) + 0.1 * gdp_growth__{country}',replacements=repl)

In [29]:
# --- Block 2: Expected loss calculation ---
expected_loss_block = Mexplode(lists + 
'>doable <> el__dim = exposure__dim * pd__dim * lgd__dim',replacements=repl)

In [30]:
# --- Block 3: Expected loss calculation per bank ---
bank_block = Mexplode(lists +    
    '''
>    do banks
>      Expected_loss_{banks} = sum(country, sum(ports, el__dim))
>    enddo   
    '''
    ,replacements=repl)

In [31]:
# --- Combine all blocks into one model ---
credit_risk_model = sum([pd_update_block + expected_loss_block + bank_block])
credit_risk_model.show

FRML <> PD__DANSKE__DENMARK__HOUSEHOLDS = 0.02+0.8*PD__DANSKE__DENMARK__HOUSEHOLDS(-1)+0.1*GDP_GROWTH__DENMARK $
FRML <> PD__DANSKE__DENMARK__NFC = 0.02+0.8*PD__DANSKE__DENMARK__NFC(-1)+0.1*GDP_GROWTH__DENMARK $
FRML <> PD__DANSKE__DENMARK__RE = 0.02+0.8*PD__DANSKE__DENMARK__RE(-1)+0.1*GDP_GROWTH__DENMARK $
FRML <> PD__DANSKE__DENMARK__SOV = 0.02+0.8*PD__DANSKE__DENMARK__SOV(-1)+0.1*GDP_GROWTH__DENMARK $
FRML <> PD__DANSKE__SWEDEN__HOUSEHOLDS = 0.02+0.8*PD__DANSKE__SWEDEN__HOUSEHOLDS(-1)+0.1*GDP_GROWTH__SWEDEN $
FRML <> PD__DANSKE__SWEDEN__NFC = 0.02+0.8*PD__DANSKE__SWEDEN__NFC(-1)+0.1*GDP_GROWTH__SWEDEN $
FRML <> PD__DANSKE__SWEDEN__RE = 0.02+0.8*PD__DANSKE__SWEDEN__RE(-1)+0.1*GDP_GROWTH__SWEDEN $
FRML <> PD__DANSKE__SWEDEN__SOV = 0.02+0.8*PD__DANSKE__SWEDEN__SOV(-1)+0.1*GDP_GROWTH__SWEDEN $
FRML <> PD__NORDEA__DENMARK__HOUSEHOLDS = 0.02+0.8*PD__NORDEA__DENMARK__HOUSEHOLDS(-1)+0.1*GDP_GROWTH__DENMARK $
FRML <> PD__NORDEA__DENMARK__NFC = 0.02+0.8*PD__NORDEA__DENMARK__NFC(-1)+0.1*GDP_GR

Of course we could also just make the model in one Mexplode or even simpler a SMexplode that was defined above.  

In [32]:
total_credit_risk_model = SMexplode('''
>doable <> pd__dim = 0.02 + 0.8 * pd__dim(-1) + 0.1 * gdp_growth__{country}
>doable <> el__dim = exposure__dim * pd__dim * lgd__dim',replacements=repl)
>do banks
>   Expected_loss_{banks} = sum(country, sum(ports, el__dim))
>enddo ''')
total_credit_risk_model.show


FRML <> PD__DANSKE__DENMARK__HOUSEHOLDS = 0.02+0.8*PD__DANSKE__DENMARK__HOUSEHOLDS(-1)+0.1*GDP_GROWTH__DENMARK $
FRML <> PD__DANSKE__DENMARK__NFC = 0.02+0.8*PD__DANSKE__DENMARK__NFC(-1)+0.1*GDP_GROWTH__DENMARK $
FRML <> PD__DANSKE__DENMARK__RE = 0.02+0.8*PD__DANSKE__DENMARK__RE(-1)+0.1*GDP_GROWTH__DENMARK $
FRML <> PD__DANSKE__DENMARK__SOV = 0.02+0.8*PD__DANSKE__DENMARK__SOV(-1)+0.1*GDP_GROWTH__DENMARK $
FRML <> PD__DANSKE__SWEDEN__HOUSEHOLDS = 0.02+0.8*PD__DANSKE__SWEDEN__HOUSEHOLDS(-1)+0.1*GDP_GROWTH__SWEDEN $
FRML <> PD__DANSKE__SWEDEN__NFC = 0.02+0.8*PD__DANSKE__SWEDEN__NFC(-1)+0.1*GDP_GROWTH__SWEDEN $
FRML <> PD__DANSKE__SWEDEN__RE = 0.02+0.8*PD__DANSKE__SWEDEN__RE(-1)+0.1*GDP_GROWTH__SWEDEN $
FRML <> PD__DANSKE__SWEDEN__SOV = 0.02+0.8*PD__DANSKE__SWEDEN__SOV(-1)+0.1*GDP_GROWTH__SWEDEN $
FRML <> PD__NORDEA__DENMARK__HOUSEHOLDS = 0.02+0.8*PD__NORDEA__DENMARK__HOUSEHOLDS(-1)+0.1*GDP_GROWTH__DENMARK $
FRML <> PD__NORDEA__DENMARK__NFC = 0.02+0.8*PD__NORDEA__DENMARK__NFC(-1)+0.1*GDP_GR

And the causal structure can be displayed

In [33]:
credit_risk_model.draw